## Runs DE on different clusters. Also runs DE within each cluster for each condition

In [1]:
# Input info
# integrated_f = "/data/isshamie/mito_lineage/output/annotation/TcellDupi_may17_2021/MTblacklist/mergedSamples/allSamples.integrated.rds"
# outdir = "/data/isshamie/mito_lineage/output/annotation/TcellDupi_may17_2021/MTblacklist/mergedSamples/DE" 
se_f = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/annotation_clones/SE.rds"
sample_names = "preB,postB"
outdir = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/annotation_clones/peaks_donors_DE_minPct_00"

# Parameters
nTop = 25000
assay="ATAC"

minPct = 0.00
cores = 4
p_thresh=0.1
utils_script <- "/data/Mito_Trace/R_scripts/seurat_utils.R"

comps_f = ""
#cond.comparisons <- "A,preA,postA;B,preB,postB"

In [2]:
source(utils_script)

Attaching SeuratObject



In [3]:
#clust_atac_outdir = file.path(outdir, "clusters_atac")
dir.create(outdir, showWarnings = FALSE)
#dir.create(outdir, showWarnings = FALSE)

In [4]:
sample_names <- unlist(strsplit(sample_names, ",")[[1]])
#samples <- unlist(strsplit(samples, ","))

sample_names

[1] "preB"  "postB"

In [5]:
library(GenomicRanges)
library(Seurat)
library(Signac)
library(GenomeInfoDb)
library(EnsDb.Hsapiens.v75)
library(ggplot2)
library(patchwork)
set.seed(1234)
library(data.table)
library(magrittr)
library(cowplot)
library(metap)
library("RColorBrewer")

library(future)
plan()

plan("multiprocess", workers = cores)
#options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM
options(future.globals.maxSize = 8000 * 1024^2)

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb

Loading required package: ensembldb

Loading required package: GenomicFeatures

Loading required package: AnnotationDbi

Lo

sequential:
- args: function (..., envir = parent.frame())
- tweaked: FALSE
- call: NULL

Warning message:
"Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead, explicitly specify either 'multisession' or 'multicore'. In the current R session, 'multiprocess' equals 'multicore'."


In [6]:
se <- readRDS(se_f)#file.path(indir, paste0("allSamples.integrated.rds")))
DefaultAssay(se) <- "ATAC"
se

## Stimulus specific response in each cluster for each donor

----

In [27]:
closest_genes_response <- ClosestFeature(se, regions = rownames(head(GetAssayData(se),3)))
closest_genes_response

,tx_id,gene_name,gene_id,gene_biotype,type,closest_region,query_region,distance
,<chr>,<chr>,<chr>,<chr>,<fct>,<chr>,<chr>,<int>
ENSE00001697423,ENST00000424587,AP006222.2,ENSG00000228463,lincRNA,exon,chr1-267096-267253,chr1-268046-268142,792
ENST00000610067,ENST00000610067,LINC01128,ENSG00000228794,processed_transcript,gap,chr1-764485-783033,chr1-778355-779233,0
ENST00000594233,ENST00000594233,AL645608.2,ENSG00000269308,protein_coding,cds,chr1-819961-819983,chr1-827080-827940,7096


## Each donor separately

In [15]:
source(utils_script)
clust.ids <- levels(se$seurat_clusters)
don.ids <- levels(se$donor)
all.pvals <- data.frame(matrix(nrow=dim(se)[1],ncol=0),
                          row.names=row.names(se))
cols <- c("seurat_clusters", "donor")
curr.ids <- levels(as.factor(apply(se[[]][ , cols ] , 1 , paste , collapse = "_" )))
Idents(se) <- as.factor(apply(se[[]][ , c("orig.ident", cols) ] , 1 , paste , collapse = "_" ))
se$curr.id = Idents(se)
all.names.sig <- c()
for (cl in clust.ids){
    if (cl == "None"){
        next
    }
    for (d in don.ids){
        if (d == "None"){
            print('Skipping None')
            next
        }
        c <- paste0(cl, "_", d)
        cl.names.sig = c()
        
        try({
            print("c")
            print(c)
            id1 <- paste(sample_names[[1]], c, sep="_")
            id2 <- paste(sample_names[[2]], c, sep="_")

            
        curr.name <- c
        curr.clust.donor.outdir <- file.path(outdir, paste0("clust_", cl, "donor_", d))
        dir.create(curr.clust.donor.outdir)

#         response <- FindMarkers(
#                   object = se,
#                   ident.1 = id1,
#                   ident.2 = id2,#"CD4 Naive",
#                   min.pct = minPct,
#                   test.use = 'LR', latent.vars = 'peak_region_fragments'
#                )
        curr.sig <- find.markers.and.plot(se, id1, id2, curr.clust.donor.outdir, 
                                          curr.name=curr.name, test.use = 'LR', 
                                          latent.vars = 'peak_region_fragments',
                                          min.pct=minPct, p.thresh=p_thresh)
            
        names.sig <- rownames(curr.sig)
        all.names.sig <- c(all.names.sig, head(names.sig, 3))
        all.pvals[names.sig,curr.name] <- curr.sig$p_val_adj_BH

        })
    }
}

[1] "c"
[1] "0_0"


Warning message in dir.create(curr.clust.donor.outdir):
"'/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/annotation_clones/peaks_donors_DE_minPct_00/clust_0donor_0' already exists"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"


[1] "cleaning de"
[1] "dim response"
[1] 22428     6
[1] "response plots"
                               p_val   avg_diff pct.1 pct.2 p_val_adj
chr12-12441732-12442696 0.0001053843 -0.5567109 0.000 0.119         1
chr9-71685322-71685873  0.0002220537  0.6421324 0.108 0.000         1
chr19-55461411-55462308 0.0003260054  0.7035205 0.514 0.149         1
                        p_val_adj_BH
chr12-12441732-12442696    0.7654166
chr9-71685322-71685873     0.7654166
chr19-55461411-55462308    0.7654166
[1] "p_val"        "avg_diff"     "pct.1"        "pct.2"        "p_val_adj"   
[6] "p_val_adj_BH"
Error in FetchData(object = object, vars = features, cells = cells) : 
  None of the requested variables were found: p_val, avg_diff, pct.1, pct.2, p_val_adj, p_val_adj_BH
[1] "c"
[1] "0_1"


Warning message in dir.create(curr.clust.donor.outdir):
"'/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/annotation_clones/peaks_donors_DE_minPct_00/clust_0donor_1' already exists"


[1] "cleaning de"
[1] "dim response"
[1] 18135     6
[1] "response plots"
                               p_val   avg_diff pct.1 pct.2 p_val_adj
chr6-43181860-43182455  1.667121e-05 -0.4918883 0.022 0.125 0.7054924
chr9-37027432-37028145  2.507495e-05 -0.3280447 0.011 0.075 1.0000000
chr19-36687133-36687882 8.123891e-05 -0.6374621 0.043 0.212 1.0000000
                        p_val_adj_BH
chr6-43181860-43182455     0.2273671
chr9-37027432-37028145     0.2273671
chr19-36687133-36687882    0.3159688
[1] "p_val"        "avg_diff"     "pct.1"        "pct.2"        "p_val_adj"   
[6] "p_val_adj_BH"
Error in FetchData(object = object, vars = features, cells = cells) : 
  None of the requested variables were found: p_val, avg_diff, pct.1, pct.2, p_val_adj, p_val_adj_BH
[1] "c"
[1] "0_2"


...

Error in WhichCells.Seurat(object = x, cells = cells, idents = idents,  : 
  Cannot find the following identities in the object: preB_10_1postB_10_1
[1] "c"
[1] "10_2"


Warning message in dir.create(curr.clust.donor.outdir):
"'/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/annotation_clones/peaks_donors_DE_minPct_00/clust_10donor_2' already exists"


Error in WhichCells.Seurat(object = x, cells = cells, idents = idents,  : 
  Cannot find the following identities in the object: postB_10_2
[1] "c"
[1] "10_3"


Warning message in dir.create(curr.clust.donor.outdir):
"'/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/annotation_clones/peaks_donors_DE_minPct_00/clust_10donor_3' already exists"


Error in ValidateCellGroups(object = object, cells.1 = cells.1, cells.2 = cells.2,  : 
  Cell group 1 has fewer than 3 cells
[1] "Skipping None"


In [16]:
sig.all.ordered <- sort(rowMeans(-log10(all.pvals),na.rm=T), decreasing=T)
dot <- DotPlot(se, scale=FALSE,
               features = head(names(sig.all.ordered),20), 
               cluster.idents=T) + RotatedAxis()
dot

de.plots(se, names(sig.all.ordered), outdir, curr.name="allClusters.pvalsOrdered", max.size=20)

ERROR: Error in FetchData(object = object, vars = features, cells = cells): None of the requested variables were found: 


## Same features, but dont separate by donors

In [ ]:
Idents(se)<-se$name
sig.all.ordered <- sort(rowMeans(-log10(all.pvals),na.rm=T), decreasing=T)
dot <- DotPlot(se, scale=FALSE,
               features = head(names(sig.all.ordered),20), 
               cluster.idents=T) + RotatedAxis()
dot

de.plots(se, names(sig.all.ordered), outdir, curr.name="allClusters.combineDonors.pvalsOrdered", max.size=20)

--------

## Run DE Over each cluster (not donors)

In [ ]:
clust.outdir <- file.path(outdir, "clusters_de")
dir.create(clust.outdir)

source(utils_script)

all.pvals <- data.frame(matrix(nrow=dim(integrated)[1],ncol=0),
                          row.names=row.names(integrated))
#sample_names
cluster.ids <- sort(unique(integrated$seurat_clusters))
integrated$celltype.stim <- paste(integrated$seurat_clusters, integrated$orig.ident, sep = "_")
integrated$celltype <- integrated$seurat_clusters
Idents(integrated) <- "celltype.stim"

all.names.sig <- c()
for (curr.clust in cluster.ids){
    ident.1 = paste0(curr.clust, "_", sample_names[[1]]) 
    ident.2 = paste0(curr.clust, "_", sample_names[[2]])
    try({
        print(curr.clust)
        #se.filt <- subset(se, idents = c(ident.1,ident.2))
        #curr.donor = se.filt$donor[[1]] #strsplit(c,"_")[[2]]
        curr.name <-paste0("clust_", curr.clust)
        curr.clust.outdir <- file.path(clust.outdir, paste0("clust_", curr.clust))
        dir.create(curr.clust.outdir)

        curr.sig <- find.markers.and.plot(integrated, ident.1, ident.2, curr.clust.outdir, curr.name=curr.clust, 
                              min.pct=minPct, p.thresh=p_thresh)
        names.sig <- rownames(curr.sig)
        all.names.sig <- c(all.names.sig, head(names.sig, 3))
        
        all.pvals[names.sig,curr.clust] <- curr.sig$p_val_adj_BH

        curr_clust <- subset(integrated, seurat_clusters == curr.clust)
        avg_curr_clust <- data.frame(log1p(AverageExpression(curr_clust, verbose = FALSE)$RNA))
        avg_curr_clust$gene <- rownames(avg_curr_clust)
        p1 <- ggplot(avg_curr_clust, aes_string(sample_names[[1]],sample_names[[2]])) + geom_point() + ggtitle(paste("Cluster", curr.clust))
        p1 <- LabelPoints(plot = p1, points = rownames(head(response, n = 15)), repel = TRUE)
        plot_grid(p1)
        #write.csv(response, file=file.path(curr.clust.outdir,paste0("cluster_",c,".conditionDE.csv")))
        ggsave(file.path(curr.clust.outdir,paste0("cluster_",curr.clust,".allDonors.conditionScatter.png")))
        
        })
    }
     

all.names.sig <- unique(all.names.sig)
de.plots(integrated, all.names.sig, clust.outdir, curr.name="allDonors.allClusters", max.size=15)

In [ ]:
sig.all.ordered <- sort(rowMeans(-log10(all.pvals),na.rm=T), decreasing=T)
dot <- DotPlot(integrated, scale=FALSE,
               features = head(names(sig.all.ordered),20), 
               cluster.idents=T) + RotatedAxis()
dot

de.plots(integrated, names(sig.all.ordered), outdir, curr.name="allDonors.allClusters.pvalsOrdered", max.size=20)

In [ ]:
sig.all.ordered <- sort(rowMeans(-log10(motifs.pvals),na.rm=T), decreasing=T)
dot <- DotPlot(integrated, scale=TRUE,
               features = head(names(sig.all.ordered),20), 
               cluster.idents=T) + RotatedAxis()
dot

--------

In [ ]:
sessionInfo()